In [70]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys; sys.path.append('../')
from src.data_loader.data_loader import *
import os

## Mobility

In [6]:
df = pd.read_csv('../data/processed/od_inter_mobilities_05-20.csv')

In [33]:
df = df.melt('FIPS', var_name='Date', value_name='trips_count').sort_values(by=['FIPS', 'Date'])

In [34]:
df.to_csv('../data/processed/sg_inter_mobility.csv', index=False)

## Unprocessed mobility data

In [86]:
def get_square_Xy(df):
    # string headers -> int
    col_fips = df.drop(labels=['origin'], axis=1).columns
    df.rename(columns = {c:int(c) for c in col_fips}, inplace=True)
    # Drop columns not in the rows
    df.drop(columns=list(set(df.columns[1:]).difference(set(df['origin']))), inplace=True)
    # Drop rows not in the columns
    df = df[~df['origin'].isin(set(df['origin']).difference(set(df.columns[1:])))]
    # Sort the rows and columns in ascending oridin
    df = df[['origin'] + sorted(df.drop(columns='origin').columns)]
    df = df.sort_values(by='origin', axis=0)
    # Convert fo matrix
    X = df.drop(columns='origin').to_numpy()
    y = df['origin']

    return X,y

In [79]:
OD_dir = '/Users/rflperry/Downloads/Number of trips'

In [148]:
df = None
for f in os.listdir(OD_dir):
    if f.startswith('2020'):
        OD = pd.read_csv(OD_dir + '/' + f)
        assert(np.diff(OD.values.shape)[0] == 1)
        temp = pd.DataFrame(
            np.asarray([OD['Unnamed: 0'], np.diag(OD.values[:,1:])]).T,
            columns=['FIPS', f[:-4].replace('_', '-')]
        )
        if df is None:
            df = temp
        else:
            df = df.merge(temp, on='FIPS', how='outer')

In [172]:
df.interpolate(
    method='linear', axis=0, limit_area='inside'
).dropna(
    axis=0
).melt(
    'FIPS', var_name='Date', value_name='trips_count'
).sort_values(
    by=['FIPS', 'Date']
).to_csv(
    '../data/processed/sg_inter_mobility_raw.csv', index=False
)

In [175]:
df = pd.read_csv('../data/processed/sg_inter_mobility_raw.csv')

In [178]:
df[df['FIPS'] == 1001]

,FIPS,Date,trips_count
0,1001,2020-01-01,7327.0
1,1001,2020-01-02,7862.0
2,1001,2020-01-03,8493.0
3,1001,2020-01-04,8285.0
4,1001,2020-01-05,8234.0
...,...,...,...
174,1001,2020-06-23,8261.0
175,1001,2020-06-24,8109.0
176,1001,2020-06-25,8276.0
177,1001,2020-06-26,8478.0


## Bed/Hospitals

In [39]:
df = load_acute_care()

In [40]:
df.to_csv('../data/processed/acute_care.csv', index=False)

## 